In [4]:
import pandas as pd 
import numpy as np

from sklearn import linear_model 
from sklearn import ensemble
from sklearn import preprocessing 
from sklearn import tree 
from sklearn import ensemble
from sklearn import metrics 
from sklearn import model_selection
from sklearn.model_selection import train_test_split 

import optuna
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import hyperopt

In [2]:
data=pd.read_csv(r"C:/Users/влад/Downloads/_train_sem09__1_/_train_sem09 (1).csv")
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


**Логистическая регрессия**

In [3]:
X=data.drop(columns='Activity')
y=data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

GridSearchCV


In [7]:
param_grid={
    'penalty':['l1', 'l2', 'none'],
    'solver' :['lbfgs', 'liblinear', 'sag'],
    'C' :[ 0.1, 0.3, 0.5, 0.7, 0.9]
}

grid_search_1=GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

%time grid_search_1.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_1.score(X_test, y_test)))
y_test_pred = grid_search_1.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
75 fits failed out of a total of 225.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model

Wall time: 41.5 s
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'sag'}


C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


В сумме 225 обученных моделей с 5 фолдами кросс-валидации, так что по сути их было 45. Однако из-за того, что некоторые функции ошибки не работают с некоторыми алгоритмами оптимизации, всего вышло не 225, а 150. 

RandomSearchCV

In [45]:
param_distributions={'penalty': ['l2', 'none','l1'],
              'solver': ['lbfgs', 'sag','liblinear'],
               'C': list (np.linspace(0.001, 0.5, 10, dtype=float))}

In [46]:
random_search=RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
) 

%time random_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\

Wall time: 13.1 s
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.05644444444444444}


C:\Users\влад\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Немного поэксперементировав с параметрами я обнаружил, что если, что есть разные комбинации, которые на таком небольшом количестве комбинаций стремятся к одинаковым метрикам по f1 на тестовой выборке, но для некоторых комбинаций функций ошибки и алгоритмов оптимизации эти значения находятся достаточно далеко друг от друга, поэтому в идеале на подбор комбинаций значений штрафа я бы уделил больше 10 значений. 

**Cлучайный лес**

HyperOpt

In [51]:
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
space={
    'n_estimators': hp.quniform('n_estimators', 100, 300, 10),
    'max_depth' : hp.quniform('max_depth', 15, 40, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 7, 1)
    }
       

In [56]:
random_state = 42

def hyperopt_lc(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    model.fit(X, y)
    
    score = model_selection.cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    
    return -score

In [57]:
%time

trials = Trials() 

best=fmin(hyperopt_lc, 
          space=space,
          algo=tpe.suggest, 
          max_evals=50, 
          trials=trials,
          rstate=np.random.RandomState(random_state)
         )
print("Наилучшие значения гиперпараметров {}".format(best))

Wall time: 0 ns
100%|██████████| 50/50 [05:19<00:00,  6.39s/trial, best loss: -0.8134764336673482]
Наилучшие значения гиперпараметров {'max_depth': 25.0, 'min_samples_leaf': 3.0, 'n_estimators': 270.0}


Optuna


In [59]:
def optuna_rf(trial):
  
  n_estimators = trial.suggest_int('n_estimators', 150, 400, 10)
  max_depth = trial.suggest_int('max_depth', 10, 35, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)

  
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score  

In [60]:
%%time

study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")

study.optimize(optuna_rf, n_trials=50)

[I 2022-08-14 23:40:44,905] A new study created in memory with name: RandomForestClassifier
[I 2022-08-14 23:40:48,358] Trial 0 finished with value: 0.9196319018404907 and parameters: {'n_estimators': 330, 'max_depth': 12, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9196319018404907.
[I 2022-08-14 23:40:50,804] Trial 1 finished with value: 0.9462759462759464 and parameters: {'n_estimators': 210, 'max_depth': 27, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.9462759462759464.
[I 2022-08-14 23:40:55,022] Trial 2 finished with value: 0.9460858970453853 and parameters: {'n_estimators': 350, 'max_depth': 18, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.9462759462759464.
[I 2022-08-14 23:40:59,297] Trial 3 finished with value: 0.9374427830332622 and parameters: {'n_estimators': 390, 'max_depth': 14, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.9462759462759464.
[I 2022-08-14 23:41:03,022] Trial 4 finished with value: 0.958142377024137 and parameters: {'n_e

Wall time: 2min 27s


In [61]:
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

f1_score на обучающем наборе: 0.98


Модель случайного леса на параметрах, подобранных алгоритмом optuna

In [66]:
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.81      0.76      0.78       344
           1       0.80      0.85      0.83       407

    accuracy                           0.81       751
   macro avg       0.81      0.80      0.80       751
weighted avg       0.81      0.81      0.80       751



Модель логистической регрессии на параметрах, подобранных RandomSearch алгоритмом.

In [67]:
model_lr=linear_model.LogisticRegression(**random_search.best_params_, random_state=random_state, max_iter=1000)

model_lr.fit(X_train, y_train)
y_test_preds=model_lr.predict(X_test)

print(metrics.classification_report(y_test, y_test_preds))

              precision    recall  f1-score   support

           0       0.77      0.72      0.74       344
           1       0.77      0.82      0.79       407

    accuracy                           0.77       751
   macro avg       0.77      0.77      0.77       751
weighted avg       0.77      0.77      0.77       751

